# Creo un nuovo dataset 


---
## Analisi dei commenti di un ristorante

### RoadHouse grill - Padova

URL di riferimento: https://www.tripadvisor.it/Restaurant_Review-g187867-d2104396-Reviews-Roadhouse_Grill-Padua_Province_of_Padua_Veneto.html

Home --> Ristoranti --> Padova

In [1]:
# IMPORT NECESSARI
import requests
from bs4 import BeautifulSoup

print("Imported")

Imported


In [2]:
page_URL = "https://www.tripadvisor.it/Restaurant_Review-g187867-d2104396-Reviews-Roadhouse_Grill-Padua_Province_of_Padua_Veneto.html"

data = requests.get(page_URL)
clear_data = BeautifulSoup(data.content, "lxml")

print("Done")

Done


---
### STRUTTURA HTML DEL SITO 

#### REVIEW
`<p class="partial_entry"> qui c'è la review </p>`

#### N° TOTALE DI REVIEWS
`<a data-page-number="25" ...>
    25
 </a>`
 
#### NAMES
`<span class="expand_inline scrname" …>AlexandraCapeTown</span>`

#### RATINGS
`<span class="ui_bubble_rating bubble_30"></span>`

---
### PRIMO SCRAPING (sulla prima pagina)

Ottengo i nomi, i commenti, le review e da dove è stato scritto il commento per quanto rigurda la prima pagina. Ciò è necessario, in quanto, dalla seconda in poi l'URL cambia, permettendoci di iterare.

In [3]:
# OTTENGO UNA LISTA DI DIV CHE CONTENGONO NOME, RECENSIONE E VOTO

all_reviews = []
all_names = []
all_ratings = []
all_mobile = []

review_container = clear_data.findAll("div", attrs={"class":"review-container"})

for container in review_container:
  
    ### REVIEW ###
    review = container.find('p', attrs={'class': 'partial_entry'})
    if review:
        all_reviews.append(review.text)
        
    ### NAME ###
    name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
    if name:
      all_names.append(name.text)
    else:
      all_names.append('NO_NAME')
    
    ### RATING ###
    rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
    all_ratings.append(rating['class'][1][7:8])
    
    ### MOBILE ###
    mobile = container.find('span', attrs={'class': 'viaMobile'})
    if mobile:
      all_mobile.append('Mobile')
    else:
      all_mobile.append('PC')
    
    
################################################################################################################################
        
print("Executed")
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_ratings))

Executed
Nomi:  10
Commenti:  10
Voti:  10
Scritto da:  10


### SECONDO SCRAPING (dalla seconda pagina in poi)

Notiamo dall'URL che è possibile iterare sulle pagine, ottenendo i dati!

In [4]:
from tqdm import tqdm

URL_COMPLETO = 'https://www.tripadvisor.it/Restaurant_Review-g187867-d2104396-Reviews-Roadhouse_Grill-Padua_Province_of_Padua_Veneto.html'

URL_p1 = 'https://www.tripadvisor.it/Restaurant_Review-g187867-d2104396-Reviews-or'
URL_p2 = '0-Roadhouse_Grill-Padua_Province_of_Padua_Veneto.html'
    
# OTTENGO I COMMENTI, NOMI E VOTI DALLA SECONDA PAGINA IN POI

total_pages = int(clear_data.select("a.pageNum.last.taLnk")[0].text) # 25 come intero

# per iterare dobbiamo partire dalla seconda pagina dei commenti, così nell'URL compaiono le cifre che ci servono
for r in tqdm(range(1,total_pages)):

    # OTTENGO L'URL DELLA PAGINA
    URL_temp = URL_p1 + str(r) + URL_p2

    # OTTENGO I DATI E CI APPLICO BEATIFULSOUP
    data = requests.get(URL_temp)
    clear_data = BeautifulSoup(data.content, "lxml")
    

    review_container = clear_data.findAll("div", attrs={"class":"review-container"})

    for container in review_container:
  
      ### REVIEW ###
      review = container.find('p', attrs={'class': 'partial_entry'})
      if review:
          all_reviews.append(review.text)

      ### NAME ###
      name = container.find('span', attrs={'class': ('expand_inline', 'scrname')})
      if name:
        all_names.append(name.text)
      else:
        all_names.append('NO_NAME')

      ### RATING ###
      rating = container.find('span', attrs={'class': 'ui_bubble_rating'})
      all_ratings.append(rating['class'][1][7:8])

      ### MOBILE ###
      mobile = container.find('span', attrs={'class': 'viaMobile'})
      if mobile:
        all_mobile.append('Mobile')
      else:
        all_mobile.append('PC')
    
        
################################################################################################################################
        
print("Executed")
print('total_pages: ', total_pages)
print('Nomi: ', len(all_names))
print('Commenti: ', len(all_reviews))
print('Voti: ', len(all_ratings))
print('Scritto da: ', len(all_mobile))

100%|██████████| 60/60 [01:04<00:00,  1.03it/s]

Executed
total_pages:  61
Nomi:  602
Commenti:  602
Voti:  602
Scritto da:  602


In [5]:
# display(all_reviews)

In [6]:
# print(all_names)

In [7]:
#print(all_ratings, '\n')
#print(len(all_ratings))

In [8]:
#display(all_mobile)

#### Problema recensioni troppo lunghe
Alla fine di alcune recensioni troviamo un '...Più', che non possiamo espandere. Data la mole di dati, decidiamo dunque di non considerarle.

In [9]:
all_reviews_filtered = []
counter_reviews_scartate = 0;
counter_reviews_da_usare = 0;

for review in all_reviews:
    if '...Più' in review:
        all_reviews_filtered.append('RECENSIONE_TROPPO_LUNGA')
        counter_reviews_scartate+=1;
    else:
        all_reviews_filtered.append(review)
        counter_reviews_da_usare+=1;
        
        
print(
    'Totale recensioni:', counter_reviews_scartate + counter_reviews_da_usare, '\n',
    'Recensioni analizzabili:', counter_reviews_da_usare, '\n',
    'Recensioni scartate:', counter_reviews_scartate, '\n')

display(all_reviews_filtered[:10])

Totale recensioni: 602 
 Recensioni analizzabili: 241 
 Recensioni scartate: 361 



['RECENSIONE_TROPPO_LUNGA',
 'Visto che la domenica sera a Padova tutti i ristoranti il 23 dicembre sembran chiusi,non rimando che cenare qui,almeno si ha una piccola certezza.\nStaff molto gentile.\nMolto meno il cassiere che pur di non accettare American express inventa ogni scusa.',
 'Ci siamo stati per cena. Il cibo direi niente di speciale...diciamo "nella media" ma la carne che dovrebbe essere il loro pezzo forte secondo me ha perso molto di qualità...un po\' deluso. Poi fatalità avevano finito insalate, verdure ecc... mah',
 "Sono stata in questo roadhouse per il cenone di capodanno, scegliendo del menù che avevano proposto per l'occasione. Locale grande e accogliente, personale gentile e cibo buono. Lo consiglio!",
 'RECENSIONE_TROPPO_LUNGA',
 'Ottimo il mangiare, menù vari e gustosi.\nPersonale cortese e disponibile per ogni esigenza.\nPrezzi contenuti. Consiglio per gli amanti della carne',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 'RECENSIONE_TROPPO_LUNGA',
 '

---
### Analisi dei nomi
Cerchiamo di capire se chi ha commentato è un uomo (M) o una donna (F).

#### Elimino spazi e numeri
Ho bisogno di tenere solo le informazioni riguardanti il nome, elimino il resto.

In [10]:
import re

all_names_filtered = []

regex = re.compile('[^a-zA-Z]')

for name in all_names:
    
    if ' ' in name:                           # se c'è uno spazio nel nome
        space = name.index(' ')               # ne trovo l'indice
        name = regex.sub('', name[:space])    # effettuo un controllo su tutto ciò che non è alfabeto fino a quell'indice
        all_names_filtered.append(name)       # e aggiungo la stringa
        
    else:
        name = regex.sub('', name)            # in caso contrario effettuo un controllo su tutto ciò che non è alfabeto
        all_names_filtered.append(name)       # e aggiungo la stringa
        
print(all_names, '\n')
print(all_names_filtered)

['filippot315', 'claudiot746', 'christianf97', 'Reddy95', 'Alessandro R', 'davidepA7104IS', 'Norien394', 'Gingerex', 'Voyager22625929988', 'DamReviewer', 'Raore', 'jacopoc81', 'jeep82renegade', 'Antonio D', 'Alessia H', 'siluriano', 'piereteco', 'Enrico3533', 'Fabio L', 'PDF7609', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'antgrasanto', 'ariannarodella', 'Sarawanderlust91', 'Ataru_Tomobiki', 'swiffer60', 'FabioC136', 'StefanoITALY-PN', 'francobonetto', 'andrea t', '2013g_i', 'L9416AYgiorgioc', 'giuseppev988', 'martina1985marti2018', 'Riccardo D', 'distalbere66', 'viaggia61', 'A6053TFmonicar', 'francylukasik', 'salvat7307', 'Max M', 'Dem82pd', 'Giulisch92', 'plutonuvola', 'marty1190', 'Nicolavolpatozm', 'Gianmarco_18', 'Federica S', 'Massimo M', 'bombettaPadova', 'Almacast', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'NO_NAME', 'AcedaPordenone', 'corrado v', 'L22F'

#### Utilizzo gender_guesser
Infine, posso utilizzare la libreria gender_guesser.

NB: Genderize ti permette di analizzare 1000 nomi al giorno.

In [11]:
!pip install gender_guesser

In [12]:
import gender_guesser.detector as gender
d = gender.Detector()

names_with_gender = []

for name in all_names_filtered:
    temp_gender = d.get_gender(name)
    names_with_gender.append(temp_gender)
  
print(names_with_gender)

['unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'unknown', 'unknown', 'male', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'female', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'female', 'unknown', 'unknown', 'female', 'male', 'unknown', 'female', 'female', 'unknown', 'female', 'unknown', 'unknown', 'unknown', 'unknown', 'male', 'unknown', 'unknown', 'unknown', 'unknown', 'unknown', 'u

In [13]:
all_genders = []
i = 0

for guess in names_with_gender:
    if guess == 'male' or guess == 'mostly_male':
        all_genders.append('M')
    elif guess == 'female' or guess == 'mostly_female':
        all_genders.append('F')
    else:
        all_genders.append('UNKNOWN')

    
print(all_genders)

['UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'UNKNOWN', 'UNKNOWN', 'M', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'F', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'F', 'M', 'UNKNOWN', 'F', 'F', 'UNKNOWN', 'F', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'M', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', 'UNKNOWN', '

---
## Salviamo ora i dati ottenuti in una tabella Excel
Così non dovremo ogni volta rieseguire tutto il codice



### Controlliamo che ci siano lo stesso numero di dati
Potremo così creare una tabella sfruttando Pandas

In [14]:
print('Nomi: ', len(all_names_filtered))
print('Commenti: ', len(all_reviews_filtered))
print('Voti: ', len(all_ratings))
print('Mobile: ', len(all_mobile))

Nomi:  602
Commenti:  602
Voti:  602
Mobile:  602


### Eliminiamo le recensioni troppo lunghe e quelle di cui non sappiamo il genere del commentatore
Creo una lista di liste [ ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ['NOME', 'GENERE', 'RECENSIONE', 'VOTO'], ... ]

In [15]:
complete_list = []

i = len(all_names_filtered)

for indice in range(i):
  if all_genders[indice] != 'UNKNOWN' and all_reviews_filtered[indice] != 'RECENSIONE_TROPPO_LUNGA':
    temp = [all_names_filtered[indice], all_genders[indice], all_reviews_filtered[indice], all_ratings[indice], all_mobile[indice]]
    complete_list.append(temp)
    
print('DATI ANALIZZABILI: ', len(complete_list))
display(complete_list[:5])


DATI ANALIZZABILI:  56


[['Antonio',
  'M',
  'Classica catena dal sapore stelle e strisce, piatti buoni ma non eccezionali , prezzi di mercato , servizio e atmosfera buoni per il genere di ristorante .',
  '4',
  'PC'],
 ['Alessia',
  'F',
  "cibo buonissimo, dai piatti principali, ai contorni, ai dolci. prezzi nella norma per la qualità offerta. Chi si lamenta dei prezzi elevati sembra non li legga al momento dell'ordinazione, magari leggete i prezzi prima di ordinare, altrimenti che vi lamentate a fare",
  '5',
  'PC'],
 ['Fabio',
  'M',
  'Posto ideale per stare in compagnia, i piatti sono abbondati, si mangia normale, niente di speciale, prezzi medio alti, costine buone.',
  '3',
  'Mobile'],
 ['Riccardo',
  'M',
  'Ritardo di 20 minuti nonostante la prenotazione ,camerieri che commentano la partita alla televisione con frasi scurrili al posto che prendere gli ordini. Cameriere che parlano tra di loro con la fila di gente che ha prenotato.',
  '1',
  'Mobile'],
 ['Cristina',
  'F',
  'Qualità della carne

### Creiamo la tabella

In [16]:
# !pip install pandas
# !pip install xlsxwriter
import pandas as pd

In [17]:
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'Nomi': [elemento[0] for elemento in complete_list], 
                   'Genere': [elemento[1] for elemento in complete_list], 
                   'Recensioni': [elemento[2] for elemento in complete_list],
                   'Voti': [elemento[3] for elemento in complete_list],
                   'Scritta da': [elemento[4] for elemento in complete_list]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('RISTORANTE_19.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [18]:
partial_dataframe = pd.read_excel("RISTORANTE_19.xlsx")

display(partial_dataframe)

,Nomi,Genere,Recensioni,Voti,Scritta da
0,Antonio,M,"Classica catena dal sapore stelle e strisce, p...",4,PC
1,Alessia,F,"cibo buonissimo, dai piatti principali, ai con...",5,PC
2,Fabio,M,"Posto ideale per stare in compagnia, i piatti ...",3,Mobile
3,Riccardo,M,Ritardo di 20 minuti nonostante la prenotazion...,1,Mobile
4,Cristina,F,"Qualità della carne e cottura scarsa, sia io c...",2,Mobile
5,Nicky,M,"Buon locale. Location 4 su 5, menú 4 su 5, ser...",4,Mobile
6,Elsa,F,"Locale un po' anonimo,cibo buono,come i panini...",3,Mobile
7,Anna,F,"ci ricordiamo di prenotare, anche magari 20 mi...",3,PC
8,Massimo,M,Non ci si può lamentare\nServizio buono e velo...,3,Mobile
9,Marco,M,"Ci sono stato molte volte ormai, ci torno perc...",4,Mobile
